In [1]:
# get data lists
#from nipype.interfaces.ants import ANTS
#from nipype.interfaces import fsl
import nipype.interfaces.io as nio
# regressor import
ds_conf = nio.DataGrabber()
ds_conf.inputs.base_directory = "/data" # database
ds_conf.inputs.template = "sub-*/func/*_run-1_desc-confounds_regressors.tsv" # from cwd
ds_conf.inputs.sort_filelist = True
ds_conf = ds_conf.run()

# bold file import
ds_bold = nio.DataGrabber()
ds_bold.inputs.base_directory = "/data" # database
ds_bold.inputs.template = "sub-*/func/*_run-1_space-MNI152NLin6Asym_desc-preproc_bold.nii.gz" # from cwd
ds_bold.inputs.sort_filelist = True
ds_bold = ds_bold.run()
conf_list = ds_conf.outputs.outfiles
bold_list= ds_bold.outputs.outfiles
conf_list=conf_list[:-3]+['/data/sub-VT035/func/sub-VT035_task-rest_motion-uncorrected_run-3_desc-confounds_regressors.tsv']+conf_list[-3:]
bold_list[-4]='/data/sub-VT035/func/sub-VT035_task-rest_motion-uncorrected_run-3_space-MNI152NLin6Asym_desc-preproc_bold.nii.gz'
print(len(bold_list), len(conf_list))

42 42


In [2]:
import time
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
from nilearn import datasets, plotting
from nilearn.input_data import NiftiMapsMasker
from nilearn.connectome import ConnectivityMeasure

ROOT_DIR='/output/fmri/'
FIG_DIR=ROOT_DIR+'figs/'
#confounders_index=[0,4,14]+list(range(97,105,1))+list(range(461,485,1))
confounders=['csf', 'white_matter', 'framewise_displacement', 'a_comp_cor_00',
             'a_comp_cor_01', 'a_comp_cor_02', 'a_comp_cor_03', 'a_comp_cor_04',
             'a_comp_cor_05', 'a_comp_cor_06', 'a_comp_cor_07', 'trans_x',
             'trans_x_derivative1', 'trans_x_derivative1_power2', 'trans_x_power2',
             'trans_y', 'trans_y_derivative1', 'trans_y_power2','trans_y_derivative1_power2',
             'trans_z', 'trans_z_derivative1', 'trans_z_derivative1_power2', 'trans_z_power2', 'rot_x',
             'rot_x_derivative1', 'rot_x_power2', 'rot_x_derivative1_power2',
             'rot_y', 'rot_y_derivative1', 'rot_y_derivative1_power2', 'rot_y_power2', 'rot_z', 'rot_z_derivative1',
             'rot_z_derivative1_power2', 'rot_z_power2']

atlasS = datasets.fetch_atlas_pauli_2017() #sub-cortical parc
atlasC = datasets.fetch_atlas_msdl() # Cortical parc
# Loading atlas image stored in 'maps'
atlasS_filename = atlasS['maps']
atlasC_filename = atlasC['maps']
# Loading atlas data stored in 'labels'
labelsS = atlasS['labels']
labelsC = atlasC['labels']
coordsC = atlasC.region_coords
data_all=[]

# Processing the functional datasets
n_sub = len(bold_list)

for i_sub in range(0,n_sub,1):
    #strings
    if i_sub<18:
        str_sub = bold_list[i_sub][10:16]
    else:
        str_sub = bold_list[i_sub][10:15]
        
    bold_file = bold_list[i_sub]
    conf_file = conf_list[i_sub]
    print('processing ',str_sub, ':\n' )
    t0=time.time()
    conf_data = pd.read_csv(conf_file, delimiter='\t')
    confouders_used = conf_data[confounders]
    #confouders_used = conf_data.iloc[:,confounders_index]
    confouders_used.fillna(value=0,inplace=True)
    t1=time.time()
    maskerS = NiftiMapsMasker(maps_img=atlasS_filename, standardize=True, memory='nilearn_cache', verbose=5)
    time_seriesS = maskerS.fit_transform(bold_file, confounds=confouders_used.values)
    t1=time.time()
    print('\tPauli sub-cortical Atlas takes: ', t1-t0, ' with dim= ', time_seriesS.shape, '\n')
    maskerC = NiftiMapsMasker(maps_img=atlasC_filename, standardize=True, memory='nilearn_cache', verbose=5)
    time_seriesC = maskerC.fit_transform(bold_file, confounds=confouders_used.values)
    t2=time.time()
    print('\tGeal cortical Atlas takes: ', t2-t1, ' with dim= ', time_seriesC.shape, '\n')
    # cal conn matrices
    correlation_measureS_C = ConnectivityMeasure(kind='correlation')
    correlation_matrixS_C  = correlation_measureS_C.fit_transform([time_seriesS])[0]
    correlation_measureS_P = ConnectivityMeasure(kind='partial correlation')
    correlation_matrixS_P  = correlation_measureS_P.fit_transform([time_seriesS])[0]
    correlation_measureC_C = ConnectivityMeasure(kind='correlation')
    correlation_matrixC_C  = correlation_measureC_C.fit_transform([time_seriesC])[0]
    correlation_measureC_P = ConnectivityMeasure(kind='partial correlation')
    correlation_matrixC_P  = correlation_measureC_P.fit_transform([time_seriesC])[0]
    t3=time.time()
    print('\tAll conn matrices are calculated in: ', t3-t2)
    data={'bold':bold_file,
          'time_seriesS':time_seriesS,
          'time_seriesC':time_seriesC,
          'confounders':confouders_used,
          'correlation_matrixS_C':correlation_matrixS_C,
          'correlation_matrixS_P':correlation_matrixS_P,
          'correlation_matrixC_C':correlation_matrixC_C,
          'correlation_matrixC_P':correlation_matrixC_P}
    data_all.append(data)
    # plot and sv figs
    ## Plot Conn Mat
    fig = plt.figure(figsize=[6,5],dpi=300)
    np.fill_diagonal(correlation_matrixS_C, 0)
    plotting.plot_matrix(correlation_matrixS_C,labels=labelsS,colorbar=True,vmax=0.8,vmin=-0.8,figure=fig)
    TITLE=str_sub+'-Pauli_Subcortical_Correlation_Mat'
    plt.title(TITLE)
    fig.savefig(FIG_DIR+TITLE+'.png')
    plt.close()
    fig = plt.figure(figsize=[6,5],dpi=300)
    np.fill_diagonal(correlation_matrixS_P, 0)
    plotting.plot_matrix(correlation_matrixS_P, labels=labelsS, colorbar=True,vmax=0.8, vmin=-0.8, figure=fig)
    TITLE=str_sub+'-Pauli_Subcortical_Precision_Mat'
    plt.title(TITLE)
    fig.savefig(FIG_DIR+TITLE+'.png')
    plt.close()
    fig = plt.figure(figsize=[6,5],dpi=300)
    np.fill_diagonal(correlation_matrixC_C, 0)
    plotting.plot_matrix(correlation_matrixC_C,labels=labelsC,colorbar=True,vmax=0.8,vmin=-0.8,figure=fig)
    TITLE=str_sub+'-Geal_Cortical_Correlation_Mat'
    plt.title(TITLE)
    fig.savefig(FIG_DIR+TITLE+'.png')
    plt.close()
    fig = plt.figure(figsize=[6,5],dpi=300)
    np.fill_diagonal(correlation_matrixC_P, 0)
    plotting.plot_matrix(correlation_matrixC_P, labels=labelsC, colorbar=True,vmax=0.8, vmin=-0.8, figure=fig)
    TITLE=str_sub+'-Gael_Cortical_Precision_Mat'
    plt.title(TITLE)
    fig.savefig(FIG_DIR+TITLE+'.png')
    plt.close()
    # We threshold to keep only the 20% of edges with the highest value
    # because the graph is very dense
    TITLE=str_sub+'-Geal_Cortical_Correlation_Brain'
    plotting.plot_connectome(correlation_matrixC_C,coordsC,edge_threshold="80%", colorbar=True,
                             title=TITLE, output_file=FIG_DIR+TITLE+'.png')
    TITLE=str_sub+'-Geal_Cortical_Precision_Brain'
    plotting.plot_connectome(correlation_matrixC_P,coordsC,edge_threshold="80%", colorbar=True,
                             title=TITLE, output_file=FIG_DIR+TITLE+'.png')


Bad key "text.kerning_factor" on line 4 in
/usr/local/miniconda/lib/python3.7/site-packages/matplotlib/mpl-data/stylelib/_classic_test_patch.mplstyle.
You probably need to get an updated matplotlibrc file from
http://github.com/matplotlib/matplotlib/blob/master/matplotlibrc.template
or from the matplotlib source distribution
/usr/local/miniconda/lib/python3.7/site-packages/sklearn/model_selection/_search.py:16: DeprecationWarning: Using or importing the ABCs from 'collections' instead of from 'collections.abc' is deprecated, and in 3.8 it will stop working
  from collections import Mapping, namedtuple, defaultdict, Sequence
/usr/local/miniconda/lib/python3.7/site-packages/numpy/lib/npyio.py:2278: VisibleDeprecationWarning: Reading unicode strings without specifying the encoding argument is deprecated. Set the encoding, use None for the system default.
  output = genfromtxt(fname, **kwargs)
/usr/local/miniconda/lib/python3.7/site-packages/pandas/core/frame.py:3790: SettingWithCopyWarni

processing  FMR001 :

[NiftiMapsMasker.fit_transform] loading regions from /home/neuro/nilearn_data/pauli_2017/pauli_2017_prob.nii.gz
Resampling maps


/usr/local/miniconda/lib/python3.7/site-packages/nilearn/_utils/cache_mixin.py:296: UserWarning: memory_level is currently set to 0 but a Memory object has been provided. Setting memory_level to 1.
  warnings.warn("memory_level is currently set to 0 but "


[Memory]    1.3s, 0.0min: Loading resample_img...
________________________________________resample_img cache loaded - 0.0s, 0.0min
[Memory]    1.4s, 0.0min: Loading filter_and_extract...
__________________________________filter_and_extract cache loaded - 0.0s, 0.0min
	Pauli sub-cortical Atlas takes:  10.099340677261353  with dim=  (120, 16) 

[NiftiMapsMasker.fit_transform] loading regions from /home/neuro/nilearn_data/msdl_atlas/MSDL_rois/msdl_rois.nii
Resampling maps
[Memory]    0.1s, 0.0min: Loading resample_img...


/usr/local/miniconda/lib/python3.7/site-packages/nilearn/_utils/cache_mixin.py:296: UserWarning: memory_level is currently set to 0 but a Memory object has been provided. Setting memory_level to 1.
  warnings.warn("memory_level is currently set to 0 but "


...

[NiftiMapsMasker.fit_transform] loading regions from /home/neuro/nilearn_data/pauli_2017/pauli_2017_prob.nii.gz
Resampling maps


/usr/local/miniconda/lib/python3.7/site-packages/nilearn/_utils/cache_mixin.py:296: UserWarning: memory_level is currently set to 0 but a Memory object has been provided. Setting memory_level to 1.
  warnings.warn("memory_level is currently set to 0 but "


[Memory]    1.1s, 0.0min: Loading resample_img...
________________________________________resample_img cache loaded - 0.0s, 0.0min
________________________________________________________________________________
[Memory] Calling nilearn.input_data.base_masker.filter_and_extract...
filter_and_extract('/data/sub-VT041/func/sub-VT041_task-rest_motion-uncorrected_run-1_space-MNI152NLin6Asym_desc-preproc_bold.nii.gz', 
{ 'allow_overlap': True,
  'detrend': False,
  'dtype': None,
  'high_pass': None,
  'low_pass': None,
  'maps_img': '/home/neuro/nilearn_data/pauli_2017/pauli_2017_prob.nii.gz',
  'mask_img': None,
  'smoothing_fwhm': None,
  'standardize': True,
  't_r': None,
  'target_affine': None,
  'target_shape': None}, confounds=array([[4.772161e+02, ..., 2.159078e-06],
       ...,
       [4.773632e+02, ..., 9.884217e-05]]), dtype=None, memory=Memory(cachedir='nilearn_cache/joblib'), memory_level=1, verbose=5)
[NiftiMapsMasker.transform_single_imgs] Loading data from /data/sub-VT041/

/usr/local/miniconda/lib/python3.7/site-packages/nilearn/_utils/cache_mixin.py:296: UserWarning: memory_level is currently set to 0 but a Memory object has been provided. Setting memory_level to 1.
  warnings.warn("memory_level is currently set to 0 but "


________________________________________resample_img cache loaded - 0.2s, 0.0min
________________________________________________________________________________
[Memory] Calling nilearn.input_data.base_masker.filter_and_extract...
filter_and_extract('/data/sub-VT041/func/sub-VT041_task-rest_motion-uncorrected_run-1_space-MNI152NLin6Asym_desc-preproc_bold.nii.gz', 
{ 'allow_overlap': True,
  'detrend': False,
  'dtype': None,
  'high_pass': None,
  'low_pass': None,
  'maps_img': '/home/neuro/nilearn_data/msdl_atlas/MSDL_rois/msdl_rois.nii',
  'mask_img': None,
  'smoothing_fwhm': None,
  'standardize': True,
  't_r': None,
  'target_affine': None,
  'target_shape': None}, confounds=array([[4.772161e+02, ..., 2.159078e-06],
       ...,
       [4.773632e+02, ..., 9.884217e-05]]), dtype=None, memory=Memory(cachedir='nilearn_cache/joblib'), memory_level=1, verbose=5)
[NiftiMapsMasker.transform_single_imgs] Loading data from /data/sub-VT041/func/sub-VT041_task-rest_motion-uncorrected_run-1

In [58]:
index_PD=[0,10,18,19,20,21,22,23,25,-1] #vt11 missing
index_ET=[1,2,3, 4, 5, 6,7,8,9,11,12,13,14,15,16,17,24,34,36,38,39,40] #fmr07,20,28,33,47,78,108,
# 112,145, 189; VT1-3,41,,42,43,109 missing 33 missing from the list
index_HC=[26,27,28,29,30,31,32,33,35] #vt22 missing
Groups=[index_PD, index_ET, index_HC]
Group_names=['PD', 'ET', 'HC']
print('PD: ',len(index_PD), ', ET: ',len(index_ET), ', HC: ',len(index_HC))
#data.keys()
#['bold', 'time_seriesS', 'time_seriesC', 'confounders', 'correlation_matrixS_C', 'correlation_matrixS_P', 'correlation_matrixC_C', 'correlation_matrixC_P']

PD:  10 , ET:  22 , HC:  9


In [61]:
#Group difference
pdhc=[]; ethc=[]; pdet=[];
for i_measure in range(4):
    pdhc_tmp=all_mean[0][i_measure]-all_mean[2][i_measure]; pdhc.append(pdhc_tmp)
    ethc_tmp=all_mean[1][i_measure]-all_mean[2][i_measure]; ethc.append(ethc_tmp)
    pdet_tmp=all_mean[0][i_measure]-all_mean[1][i_measure]; pdet.append(pdet_tmp)
    if i_measure<2:
        fig = plt.figure(figsize=[6,5],dpi=300)
        plotting.plot_matrix(pdhc_tmp,labels=labelsS,colorbar=True,vmax=0.5,vmin=-0.5,figure=fig)
        TITLE='PD-HC: Pauli_Subcortical_Mean_'+corr_mat_names[i_measure]
        plt.title(TITLE)
        fig.savefig(FIG_DIR+TITLE+'.png')
        plt.close()
        fig = plt.figure(figsize=[6,5],dpi=300)
        plotting.plot_matrix(ethc_tmp,labels=labelsS,colorbar=True,vmax=0.5,vmin=-0.5,figure=fig)
        TITLE='ET-HC: Pauli_Subcortical_Mean_'+corr_mat_names[i_measure]
        plt.title(TITLE)
        fig.savefig(FIG_DIR+TITLE+'.png')
        plt.close()
        fig = plt.figure(figsize=[6,5],dpi=300)
        plotting.plot_matrix(pdet_tmp,labels=labelsS,colorbar=True,vmax=0.5,vmin=-0.5,figure=fig)
        TITLE='PD-ET: Pauli_Subcortical_Mean_'+corr_mat_names[i_measure]
        plt.title(TITLE)
        fig.savefig(FIG_DIR+TITLE+'.png')
        plt.close()
    else:
        fig = plt.figure(figsize=[6,5],dpi=300)
        plotting.plot_matrix(pdhc_tmp,labels=labelsC,colorbar=True,vmax=0.5,vmin=-0.5,figure=fig)
        TITLE='PD-HC: Geal_Cortical_Mean_'+corr_mat_names[i_measure]
        plt.title(TITLE)
        fig.savefig(FIG_DIR+TITLE+'.png')
        plt.close()
        fig = plt.figure(figsize=[6,5],dpi=300)
        plotting.plot_matrix(ethc_tmp,labels=labelsC,colorbar=True,vmax=0.5,vmin=-0.5,figure=fig)
        TITLE='ET-HC: Geal_Cortical_Mean_'+corr_mat_names[i_measure]
        plt.title(TITLE)
        fig.savefig(FIG_DIR+TITLE+'.png')
        plt.close()
        fig = plt.figure(figsize=[6,5],dpi=300)
        plotting.plot_matrix(pdet_tmp,labels=labelsC,colorbar=True,vmax=0.5,vmin=-0.4,figure=fig)
        TITLE='PD-ET: Geal_Cortical_Mean_'+corr_mat_names[i_measure]
        plt.title(TITLE)
        fig.savefig(FIG_DIR+TITLE+'.png')
        plt.close()

In [62]:
atlasC

{'maps': '/home/neuro/nilearn_data/msdl_atlas/MSDL_rois/msdl_rois.nii',
 'labels': ['L Aud',
  'R Aud',
  'Striate',
  'L DMN',
  'Med DMN',
  'Front DMN',
  'R DMN',
  'Occ post',
  'Motor',
  'R DLPFC',
  'R Front pol',
  'R Par',
  'R Post Temp',
  'Basal',
  'L Par',
  'L DLPFC',
  'L Front pol',
  'L IPS',
  'R IPS',
  'L LOC',
  'Vis',
  'R LOC',
  'D ACC',
  'V ACC',
  'R A Ins',
  'L STS',
  'R STS',
  'L TPJ',
  'Broca',
  'Sup Front S',
  'R TPJ',
  'R Pars Op',
  'Cereb',
  'Dors PCC',
  'L Ins',
  'Cing',
  'R Ins',
  'L Ant IPS',
  'R Ant IPS'],
 'region_coords': [(-53.28, -8.88, 32.36),
  (53.47, -6.49, 27.52),
  (1.18, -74.54, 10.79),
  (-45.8, -64.78, 31.84),
  (-0.2, -55.21, 29.87),
  (-0.15, 51.42, 7.58),
  (51.66, -59.34, 28.88),
  (0.41, -91.05, 1.58),
  (-1.48, -27.93, 61.5),
  (40.1, 20.96, 44.72),
  (37.83, 55.49, 1.22),
  (47.53, -52.42, 43.06),
  (62.53, -32.99, -9.14),
  (-0.91, -2.75, 6.15),
  (-41.66, -59.04, 44.61),
  (-39.04, 19.28, 43.27),
  (-40.08, 50.6